# HW 2 - Naive Bayes in Hadoop MR
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__

In the live sessions for week 2 and week 3 you got some practice designing and debugging Hadoop Streaming jobs. In this homework we'll use Hadoop MapReduce to implement your first parallelized machine learning algorithm: Naive Bayes. As you develop your implementation you'll test it on a small dataset that matches the 'Chinese Example' in the _Manning, Raghavan and Shutze_ reading for Week 2. For the main task in this assignment you'll be working with a small subset of the Enron Spam/Ham Corpus. By the end of this assignment you should be able to:
* __... describe__ the Naive Bayes algorithm including both training and inference.
* __... perform__ EDA on a corpus using Hadoop MR.
* __... implement__ parallelized Naive Bayes.
* __... constrast__ partial, unordered and total order sort and their implementations in Hadoop Streaming.
* __... explain__ how smoothing affects the bias and variance of a Multinomial Naive Bayes model.

As always, your work will be graded both on the correctness of your output and on the clarity and design of your code. __Please refer to the `README` for homework submission instructions.__ 

## Notebook Setup
Before starting, run the following cells to confirm your setup.

In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
# global vars (paths) - ADJUST AS NEEDED
JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming.jar"
HDFS_DIR = "/user/root/HW2/"
HOME_DIR = "/media/notebooks/Assignments/HW2" # FILL IN HERE eg. /media/notebooks/Assignments/HW2

In [3]:
# save path for use in Hadoop jobs (-cmdenv PATH={PATH})
from os import environ
PATH  = environ['PATH']

In [4]:
# data path
ENRON = "data/enronemail_1h.txt"

In [5]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -ls 
!hdfs dfs -mkdir {HDFS_DIR}

Found 2 items
drwxr-xr-x   - root supergroup          0 2021-06-07 23:40 HW2
drwxr-xr-x   - root supergroup          0 2021-06-01 17:59 demo3
mkdir: `/user/root/HW2': File exists


# Question 1: Hadoop MapReduce Key Takeaways.  

This assignment will be the only one in which you use Hadoop Streaming to implement a distributed algorithm. The key reason we continue to teach Hadoop streaming is because of the way it forces the programmer to think carefully about what is happening under the hood when you parallelize a calculation. This question will briefly highlight some of the most important concepts that you need to understand about Hadoop Streaming and MapReduce before we move on to Spark next week.   

### Q1 Tasks:

* __a) short response:__ What "programming paradigm" is Hadoop MapReduce based on? What are the main ideas of this programming paradigm and how does MapReduce exemplify these ideas?

* __b) short response:__ What is the Hadoop Shuffle? When does it happen? Why is it potentially costly? Describe one specific thing we can we do to mitigate the cost associated with this stage of our Hadoop Streaming jobs.

* __c) short response:__ In Hadoop Streaming why do the input and output record format of a combiner script have to be the same? [__`HINT`__ _what level of combining does the framework guarantee? what is the relationship between the record format your mapper emits and the format your reducer expects to receive?_]

* __d) short response:__ To what extent can you control the level of parallelization of your Hadoop Streaming jobs? Please be specific.

* __e) short response:__ What change in the kind of computing resources available prompted the creation of parallel computation frameworks like Hadoop? 

### Q1 Student Answers:

> __a)__ Functional programming involves splitting a problem into smaller functions that can be parrallelized on different nodes .These nodes do not communicate with each other, function in silos and avoid having a global state or variable. This type of programming is done with expressions. Hadoop mapreduce consist of high order functions like map and reduce that work using this paradigm. They accept the other function as an argument.Map phase generally runs in parallel on different chunks of data  distributed across different resources and emits an output that goes to reduce phase which aggregates the inetermidiate output of mappers. Between the output and input of these functions there is no global state or variable, that is the entire framework is stateless.

> __b)__ Hadoop shuffle is the process by which hadoop streaming performs sorting within each partition and transfers each partition of the map outputs to specific reducers so that all the same keys go to same reducer. This process happens after the mapper has produced the output and before reducer phase starts. This is the most expensive part of the framework.  It is costly because it performs merge sort on the output of mappers once all mappers have completed their task and gathers all the same keys together. This operation is of nlogn where n is the number of records that go to reducer phase. We can mitigate this cost by reducing number of records to reducer phase by using local aggregations and combiners on the mapper output.

> __c)__ Input and output of combiner script should be same format because it requires to take the output of mapper and just aggregate it to reduce the number of records that are sent to shuffle process. It does not need to perform any other calculation.  

> __d)__ User can control what computation or program can run in mapper phase in parallel but cannot control number of mappers in hadoop streaming jobs. It depends on how many blocks the data is span across.The number of mappers depends on the data block size for each node and size of data to be processed. If data is 640MB file and data block size is 128 MB then hadoop will run 5 mappers per mapReduce job.However, user can specifify number of reduce tasks in hadoop streaming job, hence number of reducers running in parallel can be controlled.

> __e)__ Memory is getting cheaper and network is becoming faster.At the same time, there are several devices and products that is making dat bigger and bigger. This prompted creation of parallel computation framwords as it has become cheaper and efficient to process data on distributed network and processes.

# Question 2: MapReduce Design Patterns.  

In the last two live sessions and in your readings from Lin & Dyer you encountered a number of techniques for manipulating the logistics of a MapReduce implementation to ensure that the right information is available at the right time and location. In this question we'll review a few of the key techniques you learned.   

### Q2 Tasks:

* __a) short response:__ What are counters (in the context of Hadoop Streaming)? How are they useful? What kinds of counters does Hadoop provide for you? How do you create your own custom counter?

* __b) short response:__ What are composite keys? How are they useful? How are they related to the idea of custom partitioning?

* __c) short response:__ What is the order inversion pattern? What problem does it help solve? How do we implement it? 

### Q2 Student Answers:

> __a)__ Whenever a hadoop streaming job gets executed, Hadoop Framework initiates Counters to keep track of job statistics like the number of bytes read, the number of rows read, the number of rows written etc after each task.This gives information to users regarding how much of data is being processed and produced by each task, which is specially useful to monitor the smount of work each reducer node is doing. It is also helful to monitor the effectiveness of comniners in reducing input to shuffle. It also gives info to confirm that the expected amount of input was processed and expected output amount of output was produced.There are some built-in Counters in Hadoop for every job. 

    1) MapReduce Task Counters: collects tasks information over the course of their execution and then aggregate results over all the tasks.
    2) File System Counter: It tracks the number of bytes read by the file system and a number of bytes written.
    3) Job Counter: Jobtracker (or application master in YARN) maintain Job Counters, so they don’t need to be sent across the network.
Custom Counters can be used to have track of any kind of statistics about the records written as logic in Mapper and Reducers.It is useful in the debugging process to find the number of Bad Records.Custom counters are a slight departure from statelessness, however in this case the framework manages so that you can increment them from any node without causing a race condition.
To use a custom counter we can write an appropriately formatted line to the standard error stream. For example the following line would increment a counter called 'counter-name' in group 'MyWordCounters' by 10:
        sys.stderr.write("reporter:counter:MyWordCounters,counter-name,10\n")


> __b)__ For complex operation,we may need to sort not just by the key but also by value.This is called a 'secondary sort'. For this, we may need to have composite key that constitute of 2 or more fields in the data and specified in hadoop streaming job to be used for sorting instead of default sorting.We can tell Hadoop that we want to make comparisons (for sorting) based on the fields in this composite keyas well as the order and the data type to consider for sorting. 
User can also define custom partitioning in mapreduce. User can include the keys of the custom partitioning in the composite key and can tell Hadoop to use that part of composite key to sort partitions. In this way, user can achieve not only sorted output within custom partition but also have the partitions in an order as well.

> __c)__ The order inversion is design pattern that is useful when we need reducers to reduce to ratio/relative frequencies or calculations of the nature that requires to get metrics based on total number of records and not just records within a key. For such calculations, we will need the output of mapper's computation in the beginning of reduce task before reducers begins to calculate. This design exploits the sorting phase of MapReduce to push data needed for calculations to the reducer ahead of the data that will be manipulated.For example, with understanding of hadoop's default sorting, the total records can be emitted by mapper as an additional key with a special character (lets say as '*total'). Since hadoop will always sort special character at the top, the output to reducers will have this additional key in the beginning itself.


# Question 3: Understanding Total Order Sort

The key challenge in distributed computing is to break a problem into a set of sub-problems that can be performed without communicating with each other. Ideally, we should be able to define an arbirtary number of splits and still get the right result, but that is not always possible. Parallelization becomes particularly challenging when we need to make comparisons between records, for example when sorting. Total Order Sort allows us to order large datasets in a way that enables efficient retrieval of results. Before beginning this assignment, make sure you have read and understand the [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb). You can skip the first two MRJob sections, but the rest of section III and all of section IV are **very** important (and apply to Hadoop Streaming) so make sure to read them closely. Feel free to read the Spark sections as well but you won't be responsible for that material until later in the course. To verify your understanding, answer the following questions.

### Q3 Tasks:

* __a) short response:__ What is the difference between a partial sort, an unordered total sort, and a total order sort? From the programmer's perspective, what does total order sort allow us to do that we can't with unordered total? Why is this important with large datasets?

* __b) short response:__ Which phase of a MapReduce job is leveraged to implement Total Order Sort? Which default behaviors must be changed. Why must they be changed?

* __c) short response:__ Describe in words how to configure a Hadoop Streaming job for the custom sorting and partitioning that is required for Total Order Sort.  

* __d) short response:__ Explain why we need to use an inverse hash code function.

* __e) short response:__ Where does this function need to be located so that a Total Order Sort can be performed?

### Q3 Student Answers:

> __a)__ In Partial Sort, the reducer output will partition file/s and each key value in partition file will be sorted based on the default key. In unordered sort, the sorting within partition files based upon a specified key. Total sort will not only have keys within each partition file sorted in specified way but will also have partition files in an order.
Total order sort allows user to define ordering of partitions Unordered sort where partition files will need to be re-stacked to generate a total ordering. We will need small post-processing step that is required to sort the partition files after the map-reduce job finishes. This is not needed in Total order sort since it allows user to define ordering of partitions in the hadoop streaming job itself. 
In unordered sort, since we dont know which partition file has top or bottom results,for large dataset it is inefficient. For big data it is important information which can avoid going through entire data. 

> __b)__ Mapping phase is used to define custom partitioning, instead of hadoop's default partitioning. However, we need to reverse engineer hadoop hash function to define partition key so that we can create partition keys in order for human reading rather than hadoop's inbuilt ordering of characters. This key needs to be specified in hadoop streaming job and we need to tell hadoop to use custom partitioning so that shuffle phase of mapreduce job can pick this key to send sorted order of partition files to reducers. If we dont specify in hadoop streaming job, hadoop will create partitions and sort with its default behaviour. That may cause imbalance in data that are sent to reducers and there may be situation where one reducer may process larger part of data than other reducers.

> __c)__ For Custom sorting, we need to  add three parameters to Hadoop Streaming command:

        -D stream.num.map.output.key.fields=n : tells Hadoop to treat first n (tab separated) fields as a composite key.

        -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator : tells Hadoop that we want to make comparisons (for sorting) based on the fields in this composite key

        -D mapreduce.partition.keycomparator.options="-ki,i -kn,n ": Tells Hadoop to perform a sort on the ith field in the composite key and then break ties by sorting on the nth field in the composite key. 

    For custom partitioning, Hadoop provides a way to partition the data across reducers in a user-defined way. This is done telling Hadoop to use all or part of the composite key as a partitioning key. All lines with the same partition key are guaranteed to be processed by the same reducer. To use custom partitioning, there are 2 more parameters we need to add to our Hadoop Streaming Command:

        -D mapreduce.partition.keypartitioner.options="-k1,1": tells Hadoop that we want to use the first key field as a partition key. We can set any key in mapper output to be set as partition key

        -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner: tells Hadoop that we want to partition the data in a custom way.

> __d)__ By default, Hadoop computes the partition index for keys produced by mappers. When we configure Hadoop Streaming to use KeyBasedPartitioner, Hadoop reads in the partition key as a string. The partition key is then hashed (as string) and hadoop uses this hashkey for sorting which is different from sorting by string.In order to preserve partition key ordering, we need to have inverse hashCode function  that takes as input the desired partition index and total number of partitions, and returns the partition key.

> __e)__ The inverse hashcode needs to be done while defining custom partitions in mapper 

# About the Data
For the main task in this portion of the homework you will train a classifier to determine whether an email represents spam or not. You will train your Naive Bayes model on a 100 record subset of the Enron Spam/Ham corpus available in the HW2 data directory (__`HW2/data/enronemail_1h.txt`__).

__Source:__   
The original data included about 93,000 emails which were made public after the company's collapse. There have been a number raw and preprocessed versions of this corpus (including those available [here](http://www.aueb.gr/users/ion/data/enron-spam/index.html) and [here](http://www.aueb.gr/users/ion/publications.html)). The subset we will use is limited to emails from 6 Enron employees and a number of spam sources. It is part of [this data set](http://www.aueb.gr/users/ion/data/enron-spam/) which was created by researchers working on personlized Bayesian spam filters. Their original publication is [available here](http://www.aueb.gr/users/ion/docs/ceas2006_paper.pdf). __`IMPORTANT!`__ _For this homework please limit your analysis to the 100 email subset which we provide. No need to download or run your analysis on any of the original datasets, those links are merely provided as context._

__Preprocessing:__  
For their work, Metsis et al. (the authors) appeared to have pre-processed the data, not only collapsing all text to lower-case, but additionally separating "words" by spaces, where "words" unfortunately include punctuation. As a concrete example, the sentence:  
>  `Hey Jon, I hope you don't get lost out there this weekend!`  

... would have been reduced by Metsis et al. to the form:  
> `hey jon , i hope you don ' t get lost out there this weekend !` 

... so we have reverted the data back toward its original state, removing spaces so that our sample sentence would now look like:
> `hey jon, i hope you don't get lost out there this weekend!`  

Thus we have at least preserved contractions and other higher-order lexical forms. However, one must be aware that this reversion is not complete, and that some object (specifically web sites) will be ill-formatted, and that all text is still lower-cased.


__Format:__   
All messages are collated to a tab-delimited format:  

>    `ID \t SPAM \t SUBJECT \t CONTENT \n`  

where:  
>    `ID = string; unique message identifier`  
    `SPAM = binary; with 1 indicating a spam message`  
    `SUBJECT = string; title of the message`  
    `CONTENT = string; content of the message`   
    
Note that either of `SUBJECT` or `CONTENT` may be "NA", and that all tab (\t) and newline (\n) characters have been removed from both of the `SUBJECT` and `CONTENT` columns.  

In [6]:
# take a look at the first 100 characters of the first 5 records (RUN THIS CELL AS IS)
!head -n 5 /media/notebooks/Assignments/HW2/{ENRON} | cut -c-100

0001.1999-12-10.farmer	0	 christmas tree farm pictures	NA
0001.1999-12-10.kaminski	0	 re: rankings	 thank you.
0001.2000-01-17.beck	0	 leadership development pilot	" sally:  what timing, ask and you shall receiv
0001.2000-06-06.lokay	0	" key dates and impact of upcoming sap implementation over the next few week
0001.2001-02-07.kitchen	0	 key hr issues going forward	 a) year end reviews-report needs generating 


In [7]:
# see how many messages/lines are in the file 
#(this number may be off by 1 if the last line doesn't end with a newline)
!wc -l /media/notebooks/Assignments/HW2/{ENRON}

100 /media/notebooks/Assignments/HW2/data/enronemail_1h.txt


In [8]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -mkdir {HDFS_DIR}

mkdir: `/user/root/HW2': File exists


In [9]:
# load the data into HDFS (RUN THIS CELL AS IS)
!hdfs dfs -copyFromLocal /media/notebooks/Assignments/HW2/{ENRON} {HDFS_DIR}/enron.txt

copyFromLocal: `/user/root/HW2/enron.txt': File exists


# Question 4:  Enron Ham/Spam EDA.
Before building our classifier, lets get aquainted with our data. In particular, we're interested in which words occur more in spam emails than in legitimate ("ham") emails. In this question you'll implement two Hadoop MapReduce jobs to count and sort word occurrences by document class. You'll also learn about two new Hadoop streaming parameters that will allow you to control how the records output from your mappers are partitioned for reducing on separate nodes. 

__`IMPORTANT NOTE:`__ For this question and all subsequent items, you should include both the subject and the body of the email in your analysis (i.e. concatetate them to get the 'text' of the document).

### Q4 Tasks:
* __a) code:__ Complete the missing components of the code in __`EnronEDA/mapper.py`__ and __`EnronEDA/reducer.py`__ to create a Hadoop  MapReduce job that counts how many times each word in the corpus occurs in an email for each class. Pay close attention to the data format specified in the docstrings of these scripts _-- there are a number of ways to accomplish this task, we've chosen this format to help illustrate a technique in `part e`_. Run the provided unit tests to confirm that your code works as expected, then run the provided Hadoop Streaming command to apply your analysis to the Enron data.


* __b) code + short response:__ How many times does the word "__assistance__" occur in each class? (`HINT:` Use a `grep` command to read from the results file you generated in '`a`' and then report the answer in the space provided.)


* __c) short response:__ Would it have been possible to add some sorting parameters to the Hadoop streaming command that would cause our `part a` results to be sorted by count? Explain why or why not. (`HINT:` This question demands an understanding of the sequence of the phases of MapReduce.)


* __d) code + short response:__ Write a second Hadoop MapReduce job to sort the output of `part a` first by class and then by count. Run your job and save the results to a local file. Then describe in words how you would go about printing the top 10 words in each class given this sorted output. (`HINT 1:` _remember that you can simply pass the `part a` output directory to the input field of this job; `HINT 2:` since this task is just reodering the records from `part a` we don't need to write a mapper or reducer, just use `/bin/cat` for both_)


* __e) code:__ A more efficient alternative to '`grep`-ing' for the top 10 words in each class would be to use the Hadoop framework to separate records from each class into its own partition so that we can just read the top lines in each. Rewrite your job from ` part d` to specify 2 reduce tasks and to tell Hadoop to partition based on the second field (which indicates spam/ham in our data). Your code should maintain the secondary sort -- that is each partition should list words from most to least frequent.

### Q4 Student Answers:
> __b)__ assistance occurs 8 times in spam and 2 times in ham class.


> __c)__ No, because we aggregate the counts in reducer and key comparator sorts mapper result  which is before reducer, so at that time we dont have the aggregated word counts.

> __d)__ we can either of below commands which will print top 10 words from the sorted output.
        !head -n 10 EnronEDA/sortresults.txt
        !grep -m 10 "" EnronEDA/sortresults.txt

In [10]:
# part a - do your work in the provided scripts then RUN THIS CELL AS IS
!chmod a+x EnronEDA/mapper.py
!chmod a+x EnronEDA/reducer.py

In [11]:
# part a - unit test EnronEDA/mapper.py (RUN THIS CELL AS IS)
!echo -e "d1	1	title	body\nd2	0	title	body" | EnronEDA/mapper.py

title	1	1
body	1	1
title	0	1
body	0	1


In [12]:
# part a - unit test EnronEDA/reducer.py (RUN THIS CELL AS IS)
!echo -e "one	1	1\none	0	1\none	0	1\ntwo	0	1" | EnronEDA/reducer.py

one	1	1
one	0	2
two	1	0
two	0	1


In [13]:
# part a - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

Deleted /user/root/HW2/eda-output


In [14]:
# part a - Hadoop streaming job (RUN THIS CELL AS IS)
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob3753571951404809239.jar tmpDir=null
21/06/08 00:56:27 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 00:56:27 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 00:56:28 INFO mapred.FileInputFormat: Total input paths to process : 1
21/06/08 00:56:28 INFO mapreduce.JobSubmitter: number of splits:2
21/06/08 00:56:28 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1623070339035_0025
21/06/08 00:56:28 INFO impl.YarnClientImpl: Submitted application application_1623070339035_0025
21/06/08 00:56:28 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1623070339035_0025/
21/06/08 00:56:28 INFO mapreduce.Job: Running job: job_1623070339035_0025
21/06/08 00:56:35 INFO mapreduce.Job: Job job_1623070339035_0025 running in uber mode : false
21/06/08 00:56:35 INFO mapreduce.Job:  map 0% reduce 0%
21/06

In [15]:
# part a - retrieve results from HDFS & copy them into a local file (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/eda-output/part-0000* > EnronEDA/results.txt

In [16]:
# part b - write your grep command here
!grep "assistance" EnronEDA/results.txt

assistance	1	8
assistance	0	2


In [17]:
# part d - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

Deleted /user/root/HW2/eda-sort-output


In [18]:
# part d - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2 -k3,3nr" \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -input {HDFS_DIR}/eda-output\
  -output {HDFS_DIR}/eda-sort-output \
  -cmdenv PATH={PATH} 
!hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-0000* > EnronEDA/sortresults.txt

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob4935332561609422204.jar tmpDir=null
21/06/08 00:56:59 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 00:56:59 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 00:57:00 INFO mapred.FileInputFormat: Total input paths to process : 2
21/06/08 00:57:00 INFO mapreduce.JobSubmitter: number of splits:2
21/06/08 00:57:00 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1623070339035_0026
21/06/08 00:57:00 INFO impl.YarnClientImpl: Submitted application application_1623070339035_0026
21/06/08 00:57:00 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1623070339035_0026/
21/06/08 00:57:00 INFO mapreduce.Job: Running job: job_1623070339035_0026
21/06/08 00:57:07 INFO mapreduce.Job: Job job_1623070339035_0026 running in uber mode : false
21/06/08 00:57:07 INFO mapreduce.Job:  map 0% reduce 0%
21/06

In [19]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

Deleted /user/root/HW2/eda-sort-output


In [20]:
# part e - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2 -k3,3nr" \
  -D mapreduce.partition.keypartitioner.options="-k2,2" \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -input {HDFS_DIR}/eda-output\
  -output {HDFS_DIR}/eda-sort-output \
  -cmdenv PATH={PATH} \
  -numReduceTasks 2

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob2128343481644461105.jar tmpDir=null
21/06/08 00:57:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 00:57:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 00:57:29 INFO mapred.FileInputFormat: Total input paths to process : 2
21/06/08 00:57:29 INFO mapreduce.JobSubmitter: number of splits:2
21/06/08 00:57:29 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1623070339035_0027
21/06/08 00:57:29 INFO impl.YarnClientImpl: Submitted application application_1623070339035_0027
21/06/08 00:57:29 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1623070339035_0027/
21/06/08 00:57:29 INFO mapreduce.Job: Running job: job_1623070339035_0027
21/06/08 00:57:36 INFO mapreduce.Job: Job job_1623070339035_0027 running in uber mode : false
21/06/08 00:57:36 INFO mapreduce.Job:  map 0% reduce 0%
21/06

In [21]:
# part e - view the top 10 records from each partition (RUN THIS CELL AS IS)
for idx in range(2):
    print(f"\n===== part-0000{idx}=====\n")
    !hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-0000{idx} | head


===== part-00000=====

the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135	
cat: Unable to write to output stream.

===== part-00001=====

the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153	
cat: Unable to write to output stream.


__Expected output:__
<table>
<th>part-00000:</th>
<th>part-00001:</th>
<tr><td><pre>
the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135
</pre></td>
<td><pre>
the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153
</pre></td></tr>
</table>

# Question 5: Counters and Combiners.
Tuning the number of mappers & reducers is helpful to optimize very large distributed computations. Doing so successfully requires a thorough understanding of the data size at each stage of the job. As you learned in the week3 live session, counters are an invaluable resource for understanding this kind of detail. In this question, we will take the EDA performed in Question 4 as an opportunity to illustrate some related concepts.

### Q5 Tasks:
* __a) short response:__ Read the Hadoop output from your job in Question 4a to report how many records are emitted by the mappers and how many records are received be the reducers. In the context of word counting what does this number represent practically?

* __b) code:__ Note that we wrote the reducer in question 4a such that the input and output record format is identical. This makes it easy to use the same reducer script as a combiner. In the space provided below, write the Hadoop Streaming command to re-run your job from question 4a with this combining added.

* __c) short response__: Report the number of records emitted by your mappers in part b and the number of records received by your reducers. Compare your results here to what you saw in part a. Explain.

* __d) short response__: Describe a scenario where using a combiner would _NOT_ improve the efficiency of the shuffle stage. Explain. [__`BONUS:`__ how does increasing the number of mappers affect the usefulness of a combiner?]

### Q5 Student Answers:
> __a)__ Map output records=31490
		Reduce input records=31490
        This number is the total number of words in the document.

> __c)__ Map output records=31490
		Combine output records=13096
		Reduce input records=13096
 Comparede to part a, the count of records being input to reducer has decreased . This is because here for each word, the combiner is aggregrating counts in each class. This reduces the mapper output and subsequently input to shuffle and reducer.

> __d)__ If we have a data that has uniue key and values, combiners will not improve because the output from mapper is alreadty at aggregated level. If we increase the number of mappers, data chunks to each mapper will be lesser. This may lead to mappers mostly having unique keys. This will reduce the effect of combiners because most of the records may already be at aggregated level.

In [22]:
# part b - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

Deleted /user/root/HW2/eda-output


In [23]:
# part b - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -combiner reducer.py\
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob7172405864879433172.jar tmpDir=null
21/06/08 00:58:23 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 00:58:23 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 00:58:24 INFO mapred.FileInputFormat: Total input paths to process : 1
21/06/08 00:58:24 INFO mapreduce.JobSubmitter: number of splits:2
21/06/08 00:58:24 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1623070339035_0028
21/06/08 00:58:24 INFO impl.YarnClientImpl: Submitted application application_1623070339035_0028
21/06/08 00:58:24 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1623070339035_0028/
21/06/08 00:58:24 INFO mapreduce.Job: Running job: job_1623070339035_0028
21/06/08 00:58:32 INFO mapreduce.Job: Job job_1623070339035_0028 running in uber mode : false
21/06/08 00:58:32 INFO mapreduce.Job:  map 0% reduce 0%
21/06

# Question 6: Document Classification Task Overview.
The week 2 assigned reading from Chapter 13 of _Introduction to Information Retrieval_ by Manning, Raghavan and Schutze provides a thorough introduction to the document classification task and the math behind Naive Bayes. In this question we'll use the example from Table 13.1 (reproduced below) to 'train' an unsmoothed Multinomial Naive Bayes model and classify a test document by hand.

<table>
<th>DocID</th>
<th>Class</th>
<th>Subject</th>
<th>Body</th>
<tr><td>Doc1</td><td>1</td><td></td><td>Chinese Beijing Chinese</td></tr>
<tr><td>Doc2</td><td>1</td><td></td><td>Chinese Chinese Shanghai</td></tr>
<tr><td>Doc3</td><td>1</td><td></td><td>Chinese Macao</td></tr>
<tr><td>Doc4</td><td>0</td><td></td><td>Tokyo Japan Chinese</td></tr>
</table>

### Q6 Tasks:
* __a) short response:__ Equation 13.3 in Manning, Raghavan and Shutze shows how a Multinomial Naive Bayes model classifies a document. It predicts the class, $c$, for which the estimated conditional probability of the class given the document's contents,  $\hat{P}(c|d)$, is greatest. In this equation what two pieces of information are required to calculate  $\hat{P}(c|d)$? Your answer should include both mathematical notatation and verbal explanation.


* __b) short response:__ The Enron data includes two classes of documents: `spam` and `ham` (they're actually labeled `1` and `0`). In plain English, explain what  $\hat{P}(c)$ and   $\hat{P}(t_{k} | c)$ mean in the context of this data. How will we would estimate these values from a training corpus? How many passes over the data would we need to make to retrieve this information for all classes and all words?


* __c) hand calculations:__ Above we've reproduced the document classification example from the textbook (we added an empty subject field to mimic the Enron data format). Remember that the classes in this "Chinese Example" are `1` (about China) and `0` (not about China). Calculate the class priors and the conditional probabilities for an __unsmoothed__ Multinomial Naive Bayes model trained on this data. Show the calculations that lead to your result using markdown and $\LaTeX$ in the space provided or by embedding an image of your hand written work. [`NOTE:` _Your results should NOT match those in the text -- they are training a model with +1 smoothing you are training a model without smoothing_]


* __d) hand calculations:__ Use the model you trained to classify the following test document: `Chinese Chinese Chinese Tokyo Japan`. Show the calculations that lead to your result using markdown and   $\LaTeX$ in the space provided or by embedding an image of your hand written work.


* __e) short response:__ Compare the classification you get from this unsmoothed model in `d`/`e` to the results in the textbook's "Example 1" which reflects a model with Laplace plus 1 smoothing. How does smoothing affect our inference?

### Q6 Student Answers:
> __a)__ The pieces we need to calculate  $\hat{P}(c|d)$ are:
         the  probablity of class $\hat{P}(c)$  and  
         the conditional probabality of each term $t_{k}$ of being in a class $\hat{P}(t_{k} | c)$. 

> __b)__  $\hat{P}(c)$  is the probablity of email being spam or ham. It is called class prior. 
 $\hat{P}(c=S)$ can be calculated as number of spam emails divided by total emails.
 $\hat{P}(c=H)$ can be calculated as number of ham emails divided by total emails.
 $\hat{P}(t_{k} | c)$  is  number of times a term is occurs in the class / total number of words in the class.It is also called likelihood
We can estimate class prior from training corpus by counting total number of documents for each class and divide by total number of documents. Likelihood for each term can be calculated by getting total number of times a term is occurs in each class and divide it by total number of words in each class 
We need 2 passes if we want to calculate  $\hat{P}(t_{k} | c)$ with +1 smoothing along with $\hat{P}(c)$  and 1 pass otherwise.

> __c)__ Class priors can be calculated as below:
$$\hat{P}(c) = \frac{N_c}{N}$$
for Chinese class:
$$\hat{P}(c=1) = \frac{N_1}{N}$$
$$= \frac{3}{4}$$
$$= 0.75$$
for Non Chinese class:
$$\hat{P}(c=0) = \frac{N_0}{N}$$
$$= \frac{1}{4}$$
$$= 0.25$$

for likelihoods of each word for chinese class:

$$\hat{P}(t_{k} | c) = \frac{T_ct}{\sum_{t ' \epsilon V}{T_ct' }}$$ 

for word 'chinese' in class 'chinese':
$$\hat{P}(t = chinese | c= chinese)$$
$$= \frac{5}{8}$$
$$= 0.625$$
for word 'chinese' in class 'non-chinese':
$$\hat{P}(t = chinese | c= non-chinese)$$
$$= \frac{1}{3}$$
$$= 0.333$$

for word 'beijing' in class 'chinese':
$$\hat{P}(t = beijing | c= chinese)$$
$$= \frac{1}{8}$$
$$= 0.125$$
for word 'beijing' in class 'non-chinese':
$$\hat{P}(t = beijing | c= non-chinese)$$
$$= \frac{0}{3}$$
$$= 0$$

for word 'Shanghai' in class 'chinese':
$$\hat{P}(t = Shanghai | c= chinese)$$
$$= \frac{1}{8}$$
$$= 0.125$$
for word 'Shanghai' in class 'non-chinese':
$$\hat{P}(t = Shanghai | c= non-chinese)$$
$$= \frac{0}{3}$$
$$= 0$$


for word 'Macao' in class 'chinese':
$$\hat{P}(t = Macao | c= chinese)$$
$$= \frac{1}{8}$$
$$= 0.125$$
for word 'Macao' in class 'non-chinese':
$$\hat{P}(t = Macao | c= non-chinese)$$
$$= \frac{0}{3}$$
$$= 0$$

for word 'tokyo' in class 'chinese':
$$\hat{P}(t = tokyo | c= chinese)$$
$$= \frac{0}{8}$$
$$= 0$$
for word 'tokyo' in class 'non-chinese':
$$\hat{P}(t = tokyo | c= non-chinese)$$
$$= \frac{1}{3}$$
$$= 0.333$$

for word 'japan' in class 'chinese':
$$\hat{P}(t = japan | c= chinese)$$
$$= \frac{0}{8}$$
$$= 0$$
for word 'japan' in class 'non-chinese':
$$\hat{P}(t = japan | c= non-chinese)$$
$$= \frac{1}{3}$$
$$= 0.333$$



> __d)__ Class priorsfor Chinese class $\hat{P}(c=chinese)$:
$$0.75$$
and for Non Chinese class:
$$0.25$$

from above calculations:
 $$\hat{P}(c= `Chinese'|d=`Chinese Chinese Chinese Tokyo Japan` )$$
 $$= \hat{P}(c=`Chinese')*\hat{P}(t = chinese | c= chinese)*\hat{P}(t = chinese | c= chinese)*\hat{P}(t = chinese | c= chinese)*\hat{P}(t = tokyo | c= chinese)*\hat{P}(t = japan | c= chinese)$$
 $$= 0.75 * 0.625* 0.625* 0.625* 0* 0$$
 $$=0$$
 
 $$\hat{P}(c= `non-Chinese'|d=`Chinese Chinese Chinese Tokyo Japan` )$$
 $$= \hat{P}(c=`non-Chinese')*\hat{P}(t = chinese | c= non-chinese)*\hat{P}(t = chinese | c= non-chinese)*\hat{P}(t = chinese | c= non-chinese)*\hat{P}(t = tokyo | c= non-chinese)*\hat{P}(t = tokyo | c= non-chinese)$$
 $$= 0.25* 0.333 * 0.333* 0.333 * 0.333* 0.333$$
 $$=0.000978$$

since $\hat{P}(c= `Chinese'|d) <\hat{P}(c= `non-Chinese'|d)$, the class of the document is non chinese

> __e)__ for document  `Chinese Chinese Chinese Tokyo Japan`, laplace smoothing will assign small probablity to tokyo and japan, hence the $\hat{P}(c= `Chinese'|d)$ is not zero and is greater than $\hat{P}(c= `non-Chinese'|d)$. So now we are able to accurately classify.


In [24]:
# part d/e - if you didn't write out your calcuations above, embed a picture of them here:
#from IPython.display import Image
#Image(filename="path-to-hand-calulations-image.png")

# Question 7: Naive Bayes Inference.
In the next two questions you'll write code to parallelize the Naive Bayes calculations that you performed above. We'll do this in two phases: one MapReduce job to perform training and a second MapReduce to perform inference. While in practice we'd need to train a model before we can use it to classify documents, for learning purposes we're going to develop our code in the opposite order. By first focusing on the pieces of information/format we need to perform the classification (inference) task you should find it easier to develop a solid implementation for training phase when you get to question 8 below. In both of these questions we'll continue to use the Chinese example corpus from the textbook to help us test our MapReduce code as we develop it. Below we've reproduced the corpus, test set and model in text format that matches the Enron data.

### Q7 Tasks:
* __a) short response:__ run the provided cells to create the example files and load them in to HDFS. Then take a closer look at __`NBmodel.txt`__. This text file represents a Naive Bayes model trained (with Laplace +1 smoothing) on the example corpus. What are the 'keys' and 'values' in this file? Which record means something slightly different than the rest? The value field of each record includes two numbers which will be helpful for debugging but which we don't actually need to perform inference -- what are they? [`HINT`: _This file represents the model from Example 13.1 in the textbook, if you're having trouble getting oriented try comparing our file to the numbers in that example._]


* __b) short response:__ When performing Naive Bayes in practice instead of multiplying the probabilities (as in equation 13.3) we add their logs (as in equation 13.4). Why do we choose to work with log probabilities? If we had an unsmoothed model, what potential error could arise from this transformation?


* __c) short response:__ Documents 6 and 8 in the test set include a word that did not appear in the training corpus (and as a result does not appear in the model). What should we do at inference time when we need a class conditional probability for this word?


* __d) short response:__ The goal of our MapReduce job is to stream over the test set and classify each document by peforming the calculation from equation 13.4. To do this we'll load the model file (which contains the probabilities for equation 13.4) into memory on the nodes where we do our mapping. This is called an in-memory join. Does loading a model 'state' like this depart from the functional programming principles? Explain why or why not. From a scability perspective when would this kind of memory use be justified? when would it be unwise?


* __e) code:__ Complete the code in __`NaiveBayes/classify_mapper.py`__. Read the docstring carefully to understand how this script should work and the format it should return. Run the provided unit tests to confirm that your script works as expected then write a Hadoop streaming job to classify the Chinese example test set. [`HINT 1:` _you shouldn't need a reducer for this one._ `HINT 2:` _Don't forget to add the model file to the_ `-files` _parameter in your Hadoop streaming job so that it gets shipped to the mapper nodes where it will be accessed by your script._]


* __f) short response:__ In our test example and in the Enron data set we have fairly short documents. Since these fit fine in memory on a mapper node we didn't need a reducer and could just do all of our calculations in the mapper. However with much longer documents (eg. books) we might want a higher level of parallelization -- for example we might want to process parts of a document on different nodes. In this hypothetical scenario how would our algorithm design change? What could the mappers still do? What key-value structure would they emit? What would the reducers have to do as a last step?

### Q7 Student Answers:
> __a)__ The key is word and the values are spam count, ham count, likelihood of each term being in spam class,likelihood of each term being in ham class 
 The record which is different is the key Classpriors. the first 2 values of class prior key are the count of words in spam and ham. These values will be helpful in debugging.

> __b)__ In Naive Bayes, a problem arises when the size of the vocabulary gets very big.Since $\hat{P}(t_{k} | c)$ is between 0 and 1,  when we start multiplying them together for each word, the overall product starts approaching zero which often cannot be represented in a double or long double. This issue caused computational issue in almost every language which uses a standard floating-point data type with a fixed number of bits . The calculated probablity will eventuallyrun out of precision with extremely tiny numbers.Work-around for this situation is to represent probabilities as log probabilities instead. This is the problem of underflow. For unsmoothened model, for new or unseen word, the probablity is going to be 0. Log transformation of 0 will lead to error.

> __c)__ We can ignore it, or add this word to both classes and then take log.

> __d)__ Technically, it is a slight departure from statelessness, however,since we have loaded it on every node and every node has its own copy, the nodes can still continue to work in silos and without communicating to each other. In other words, this is okay since there is no issue of race conditions.From scalability perpective, it is okay as long as such files are small in size since keeping them in memory is going to have fast access. But when the files become large, it is going to start takling up resources and may have to be spilled to disk, which are slower to access. It will be unwise to keep them in memory in such case.

> __e)__ Complete the coding portion of this question before answering 'f'.

> __f)__ In case when one document may not fit in one mapper, our algorithm design should produce partial probablities for the documents from each mapper. we will need to have reducer for each document to gather and reduce these partial probalities to total probabality of document. mMppers can still calculate partial probabalities. The key will be doc id but one doc id key can be on multiple mappers.The values will be true class, partial logspam and partial logham. Reducers will have to aggregate the partial log probablities , calculate pred class and emit it.


Run these cells to create the example corpus and model.

In [25]:
%%writefile NaiveBayes/chineseTrain.txt
D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese

Overwriting NaiveBayes/chineseTrain.txt


In [26]:
%%writefile NaiveBayes/chineseTest.txt
D5	1		Chinese Chinese Chinese Tokyo Japan
D6	1		Beijing Shanghai Trade
D7	0		Japan Macao Tokyo
D8	0		Tokyo Japan Trade

Overwriting NaiveBayes/chineseTest.txt


In [27]:
%%writefile NBmodel.txt
beijing	0.0,1.0,0.111111111111,0.142857142857
chinese	1.0,5.0,0.222222222222,0.428571428571
tokyo	1.0,0.0,0.222222222222,0.0714285714286
shanghai	0.0,1.0,0.111111111111,0.142857142857
ClassPriors	1.0,3.0,0.25,0.75
japan	1.0,0.0,0.222222222222,0.0714285714286
macao	0.0,1.0,0.111111111111,0.142857142857

Overwriting NBmodel.txt


In [28]:
# load the data files into HDFS
!hdfs dfs -copyFromLocal NaiveBayes/chineseTrain.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal NaiveBayes/chineseTest.txt {HDFS_DIR}

copyFromLocal: `/user/root/HW2/chineseTrain.txt': File exists
copyFromLocal: `/user/root/HW2/chineseTest.txt': File exists


Your work for `part e` starts here:

In [29]:
# part e - do your work in NaiveBayes/classify_mapper.py first, then run this cell.
!chmod a+x NaiveBayes/classify_mapper.py

In [30]:
# part e - unit test NaiveBayes/classify_mapper.py (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


In [31]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output

Deleted /user/root/HW2/chinese-output


In [32]:
# part e - write your Hadooop streaming job here
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NBmodel.txt \
  -mapper classify_mapper.py \
  -reducer /bin/cat \
  -input {HDFS_DIR}/chineseTest.txt \
  -output {HDFS_DIR}/chinese-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}


packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob9098305633850426437.jar tmpDir=null
21/06/08 01:01:40 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 01:01:40 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 01:01:41 INFO mapred.FileInputFormat: Total input paths to process : 1
21/06/08 01:01:41 INFO mapreduce.JobSubmitter: number of splits:2
21/06/08 01:01:41 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1623070339035_0029
21/06/08 01:01:41 INFO impl.YarnClientImpl: Submitted application application_1623070339035_0029
21/06/08 01:01:41 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1623070339035_0029/
21/06/08 01:01:41 INFO mapreduce.Job: Running job: job_1623070339035_0029
21/06/08 01:01:47 INFO mapreduce.Job: Job job_1623070339035_0029 running in uber mode : false
21/06/08 01:01:47 INFO mapreduce.Job:  map 0% reduce 0%
21/06

In [33]:
# part e - retrieve test set results from HDFS (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/chinese-output/part-000* > NaiveBayes/chineseResults.txt

In [34]:
# part e - take a look (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseResults.txt | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


<table>
<th> Expected output for the test set:</th>
<tr align=Left><td><pre>
d5	1	-8.90668134	-8.10769031	1
d6	1	-5.78074351	-4.17950237	1
d7	0	-6.59167373	-7.51170688	0
d8	0	-4.39444915	-5.56579673	0
</pre></td><tr>
</table>

# Question 8: Naive Bayes Training.
In Question 7 we used a model that we had trained by hand. Next we'll develop the code to do that same training in parallel, making it suitable for use with larger corpora (like the Enron emails). The end result of the MapReduce job you write in this question should be a model text file that looks just like the example (`NBmodel.txt`) that we created by hand above.

To refresh your memory about the training process take a look at  `6a` and `6b` where you described the pieces of information you'll need to collect in order to encode a Multinomial Naive Bayes model. We now want to retrieve those pieces of information while streaming over a corpus. The bulk of the task will be very similar to the word counting excercises you've already done but you may want to consider a slightly different key-value record structure to efficiently tally counts for each class. 

The most challenging (interesting?) design question will be how to retrieve the totals (# of documents and # of words in documents for each class). Of course, counting these numbers is easy. The hard part is the timing: you'll need to make sure you have the counts totalled up _before_ you start estimating the class conditional probabilities for each word. It would be best (i.e. most scalable) if we could find a way to do this tallying without storing the whole vocabulary in memory... Use an appropriate MapReduce design pattern to implement this efficiently! 


### Q8 Tasks:
* __a) make a plan:__  Fill in the docstrings for __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ to appropriately reflect the format that each script will input/output. [`HINT:` _the input files_ (`enronemail_1h.txt` & `chineseTrain.txt`) _have a prespecified format and your output file should match_ `NBmodel.txt` _so you really only have to decide on an internal format for Hadoop_].


* __b) implement it:__ Complete the code in __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ so that together they train a Multinomial Naive Bayes model __with no smoothing__. Make sure your end result is formatted correctly (see note above). Test your scripts independently and together (using `chineseTrain.txt` or test input of your own devising). When you are satisfied with your Python code design and run a Hadoop streaming command to run your job in parallel on the __chineseTrain.txt__. Confirm that your trained model matches your hand calculations from Question 6.


* __c) short response:__ We saw in Question 6 that adding Laplace smoothing (where the smoothing parameter $k=1$) makes our classifications less sensitve to rare words. However implementing this technique requires access to one additional piece of information that we had not previously used in our Naive Bayes training. What is that extra piece of information? [`HINT:` see equation 13.7 in Manning, Raghavan and Schutze].


* __d) short response:__ There are a couple of approaches that we could take to handle the extra piece of information you identified in `c`: 1) if we knew this extra information beforehand, we could provide it to our reducer as a configurable parameter for the vocab size dynamically (_where would we get it in the first place?_). Or 2) we could compute it in the reducer without storing any bulky information in memory but then we'd need some postprocessing or a second MapReduce job to complete the calculation (_why?_). Breifly explain what is non-ideal about each of these options. 


* __e) code + short response:__ Choose one of the 2 options above. State your choice & reasoning in the space below then use that strategy to complete the code in __`NaiveBayes/train_reducer_smooth.py`__. Test this alternate reducer then write and run a Hadoop streaming job to train an MNB model with smoothing on the Chinese example. Your results should match the model that we provided for you above (and the calculations in the textbook example). __IMPORTANT NOTE:__ For full credit on this question, your code must work with multiple reducers. 

    - [`HINT:` You will need to implement custom partitioning - [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb)] 

    - [`HINT:` Don't start from scratch with this one -- you can just copy over your reducer code from part `b` and make the needed modifications]. 



__IMPORTANT NOTE:__ For full credit on this question, your code must work with multiple reducers. [`HINT:`_You will need to implement custom partitioning - [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb)]


### Q8 Student Answers:
> __ c)__ The extra piece of info missing is vocabulary of the corpus, that is count of unique words in the corpus.

> __ d)__ The first option is non ideal for regular classification work where the input data is changing frequently, like reddit feed etc,  because in that case our vocab size has to be calculated everytime we run the job. In such case, its better top have this incorporated in the hadoop mapreduce job. 
On the other hand, the 2nd option will be non ideal when our feed is having more of a static vocabulary, such as weather forecast which generally has fixed/static vocabulary. Our job does not need to recalculate vocabulary everytime.

> __ e)__ I chose the 2nd option and implemented two mapreduce job because we are training model on email feed, which in practice, will have changing vocabulary size. 
    a. First mapreduce job is doing word partial counts and document partial counts in mapper and reducing/aggregating the counts in reducer with custom partitioning. The output of this job will have unique words along with aggregated document count and word count for each class.
    b. Second mapreduce is calculating vocabulary partial count using local aggregator in mapper and likelihood calculation is done in reducer. We are reusing the partition key from first job. We should keep the number of reducers for both jobs same for this reason. It is possible to decouple it by an independing custom partitioning in 2nd job. 
   Also, since 2nd mapreduce takes in aggregated result from 1st mapreduce, this strategy is more efficient than passing through entire data in both jobs. 



In [35]:
# part a - do your work in train_mapper.py and train_reducer.py then RUN THIS CELL AS IS
!chmod a+x NaiveBayes/train_mapper.py
!chmod a+x NaiveBayes/train_reducer.py
!echo "=========== MAPPER DOCSTRING ============"
!head -n 8 NaiveBayes/train_mapper.py | tail -n 6
!echo "=========== REDUCER DOCSTRING ============"
!head -n 8 NaiveBayes/train_reducer.py | tail -n 6

=========== MAPPER DOCSTRING ============
Mapper reads in text documents and emits word counts by class.
INPUT:                                                    
    DocID \t true_class \t subject \t body                
OUTPUT:                                                   
    partitionKey \t word \t class0_partialCount,class1_partialCount       
    
=========== REDUCER DOCSTRING ============
Reducer aggregates word counts by class and emits frequencies.

INPUT:
    partitionKey \t word \t class0_partialCount,class1_partialCount
OUTPUT:
    word \t class0_count,class1_count,pclass0,pclass1 


__`part b starts here`:__ MNB _without_ Smoothing (training on Chinese Example Corpus).

In [36]:
# part b - write a unit test for your mapper here
!echo -e "d1	0	hello, how are you?	hello, I am fine thankyou winner\nd2	1	lottery winner	You won $100M" | NaiveBayes/train_mapper.py|sort -k2,2

A	*total_doc_count	1,1
A	*total_word_count	10,5
A	am	1,0
A	are	1,0
A	fine	1,0
A	hello	1,0
A	hello	1,0
A	how	1,0
A	i	1,0
A	lottery	0,1
A	m	0,1
A	thankyou	1,0
A	winner	0,1
A	winner	1,0
A	won	0,1
A	you	0,1
A	you	1,0


In [37]:
# part b - write a unit test for your reducer here
!cat test_8b_reducer.txt | NaiveBayes/train_reducer.py

ClassPriors	1.0,1.0,0.5,0.5
hello	2.0,0.0,0.2,0.0
how	1.0,0.0,0.1,0.0
are	1.0,0.0,0.1,0.0
you	1.0,1.0,0.1,0.2
i	1.0,0.0,0.1,0.0
am	1.0,0.0,0.1,0.0
fine	1.0,0.0,0.1,0.0
thankyou	1.0,0.0,0.1,0.0
winner	1.0,1.0,0.1,0.2
lottery	0.0,1.0,0.0,0.2
won	0.0,1.0,0.0,0.2
m	0.0,1.0,0.0,0.2


In [38]:
# part b - write a systems test for your mapper + reducer together here
!echo -e "d1	0	hello, how are you?	hello, I am fine thankyou winner\nd2	1	lottery winner	You won $100M" | NaiveBayes/train_mapper.py|sort -k1,1 -k2,2 |NaiveBayes/train_reducer.py

ClassPriors	1.0,1.0,0.5,0.5
am	1.0,0.0,0.1,0.0
are	1.0,0.0,0.1,0.0
fine	1.0,0.0,0.1,0.0
hello	2.0,0.0,0.2,0.0
how	1.0,0.0,0.1,0.0
i	1.0,0.0,0.1,0.0
lottery	0.0,1.0,0.0,0.2
m	0.0,1.0,0.0,0.2
thankyou	1.0,0.0,0.1,0.0
winner	1.0,1.0,0.1,0.2
won	0.0,1.0,0.0,0.2
you	1.0,1.0,0.1,0.2


In [39]:
# part b - clear (and name) an output directory in HDFS for your unsmoothed chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/unsmoothed_chinese_NBmodel-output

Deleted /user/root/HW2/unsmoothed_chinese_NBmodel-output


In [40]:
# part b - write your hadoop streaming job
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer.py\
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/unsmoothed_chinese_NBmodel-output \
  -cmdenv PATH={PATH} \
  -numReduceTasks 4

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob8889737676166475622.jar tmpDir=null
21/06/08 01:02:37 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 01:02:37 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 01:02:38 INFO mapred.FileInputFormat: Total input paths to process : 1
21/06/08 01:02:38 INFO mapreduce.JobSubmitter: number of splits:2
21/06/08 01:02:38 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1623070339035_0030
21/06/08 01:02:38 INFO impl.YarnClientImpl: Submitted application application_1623070339035_0030
21/06/08 01:02:38 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1623070339035_0030/
21/06/08 01:02:38 INFO mapreduce.Job: Running job: job_1623070339035_0030
21/06/08 01:02:44 INFO mapreduce.Job: Job job_1623070339035_0030 running in uber mode : false
21/06/08 01:02:44 INFO mapreduce.Job:  map 0% reduce 0%
21/06

In [41]:
# part b - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/unsmoothed_chinese_NBmodel-output/part-000* > NaiveBayes/unsmoothed_chinese_NBmodel-output.txt

In [42]:
# part b - print your model so that we can confirm that it matches expected results
!cat NaiveBayes/unsmoothed_chinese_NBmodel-output.txt

ClassPriors	1.0,3.0,0.25,0.75
chinese	1.0,5.0,0.3333333333333333,0.625
macao	0.0,1.0,0.0,0.125
beijing	0.0,1.0,0.0,0.125
japan	1.0,0.0,0.3333333333333333,0.0
tokyo	1.0,0.0,0.3333333333333333,0.0
shanghai	0.0,1.0,0.0,0.125


__`part e starts here`:__ MNB _with_ Smoothing (training on Chinese Example Corpus).

In [43]:
# part e - write a unit test for your NEW reducer here
!chmod a+x NaiveBayes/train_second_mapper_smooth.py
!chmod a+x NaiveBayes/train_second_reducer_smooth.py
!chmod a+x NaiveBayes/train_reducer_smooth.py

!cat test_8b_reducer.txt | NaiveBayes/train_reducer_smooth.py

A	*total_doc_count	1.0,1.0
A	*total_word_count	10.0,5.0
A	hello	2.0,0.0
A	how	1.0,0.0
A	are	1.0,0.0
A	you	1.0,1.0
A	i	1.0,0.0
A	am	1.0,0.0
A	fine	1.0,0.0
A	thankyou	1.0,0.0
A	winner	1.0,1.0
A	lottery	0.0,1.0
A	won	0.0,1.0
A	m	0.0,1.0


In [44]:
# part e - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py|sort -k1,1 -k2,2 |NaiveBayes/train_reducer_smooth.py|NaiveBayes/train_second_mapper_smooth.py| sort -k1,1 -k2,2 |NaiveBayes/train_second_reducer_smooth.py

ClassPriors	1.0,3.0,0.25,0.75
beijing	0.0,1.0,0.1111111111111111,0.14285714285714285
chinese	1.0,5.0,0.2222222222222222,0.42857142857142855
japan	1.0,0.0,0.2222222222222222,0.07142857142857142
macao	0.0,1.0,0.1111111111111111,0.14285714285714285
shanghai	0.0,1.0,0.1111111111111111,0.14285714285714285
tokyo	1.0,0.0,0.2222222222222222,0.07142857142857142


In [45]:
# part e - clear (and name) an output directory in HDFS for your SMOOTHED chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/smoothed_chinese_NBmodel-output
!hdfs dfs -rm -r {HDFS_DIR}/smoothed_chinese_NBmodel-output_part1

Deleted /user/root/HW2/smoothed_chinese_NBmodel-output
Deleted /user/root/HW2/smoothed_chinese_NBmodel-output_part1


In [46]:
# part e - write your hadoop streaming job
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer_smooth.py\
  -mapper train_mapper.py \
  -reducer train_reducer_smooth.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/smoothed_chinese_NBmodel-output_part1 \
  -cmdenv PATH={PATH} \
  -numReduceTasks 4


!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -files NaiveBayes/train_second_mapper_smooth.py,NaiveBayes/train_second_reducer_smooth.py\
  -mapper train_second_mapper_smooth.py \
  -reducer train_second_reducer_smooth.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/smoothed_chinese_NBmodel-output_part1 \
  -output {HDFS_DIR}/smoothed_chinese_NBmodel-output \
  -cmdenv PATH={PATH} \
  -numReduceTasks 4

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob4567750820806118179.jar tmpDir=null
21/06/08 01:03:18 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 01:03:19 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 01:03:19 INFO mapred.FileInputFormat: Total input paths to process : 1
21/06/08 01:03:19 INFO mapreduce.JobSubmitter: number of splits:2
21/06/08 01:03:20 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1623070339035_0031
21/06/08 01:03:20 INFO impl.YarnClientImpl: Submitted application application_1623070339035_0031
21/06/08 01:03:20 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1623070339035_0031/
21/06/08 01:03:20 INFO mapreduce.Job: Running job: job_1623070339035_0031
21/06/08 01:03:26 INFO mapreduce.Job: Job job_1623070339035_0031 running in uber mode : false
21/06/08 01:03:26 INFO mapreduce.Job:  map 0% reduce 0%
21/06

In [47]:
# part e - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/smoothed_chinese_NBmodel-output/part-000* > NaiveBayes/smoothed_chinese_NBmodel-output.txt

In [48]:
# part e - print your model (above local file) so that we can confirm that it matches expected results
!cat  NaiveBayes/smoothed_chinese_NBmodel-output.txt

ClassPriors	1.0,3.0,0.25,0.75
chinese	1.0,5.0,0.2222222222222222,0.42857142857142855
macao	0.0,1.0,0.1111111111111111,0.14285714285714285
beijing	0.0,1.0,0.1111111111111111,0.14285714285714285
japan	1.0,0.0,0.2222222222222222,0.07142857142857142
tokyo	1.0,0.0,0.2222222222222222,0.07142857142857142
shanghai	0.0,1.0,0.1111111111111111,0.14285714285714285


# Question 9: Enron Ham/Spam NB Classifier & Results.

Fantastic work. We're finally ready to perform Spam Classification on the Enron Corpus. In this question you'll run the analysis you've developed, report its performance.

### Q9 Tasks:
* __a) train/test split:__ Run the provided code to split our Enron file into a training set and testing set then load them into HDFS. [`NOTE:` _Make sure you re calculate the vocab size for just the training set!_]

* __b) train 2 models:__ Write Hadoop Streaming jobs to train MNB Models on the training set with and without smoothing. Save your models to local files at __`NaiveBayes/Unsmoothed/NBmodel.txt`__ and __`NaiveBayes/Smoothed/NBmodel.txt`__. [`NOTE:` _This naming is important because we wrote our classification task so that it expects a file of that name... if this inelegance frustrates you there is an alternative that would involve a few adjustments to your code [read more about it here](http://www.tnoda.com/blog/2013-11-23)._] Finally run the checks that we provide to confirm that your results are correct.


* __c) code:__ Recall that we designed our classification job with just a mapper. An efficient way to report the performance of our models would be to simply add a reducer phase to this job and compute precision and recall right there. Complete the code in __`NaiveBayes/evaluation_reducer.py`__ and then write Hadoop jobs to evaluate your two models on the test set. Report their performance side by side. [`NOTE:` if you need a refresher on precision, recall and F1-score [Wikipedia](https://en.wikipedia.org/wiki/F1_score) is a good resource.]


* __d) short response:__ Compare the performance of your two models. What do you notice about the unsmoothed model's predictions? Can you guess why this is happening? Which evaluation measure do you think is most relevant in our use case? [`NOTE:` _Feel free to answer using your common sense but if you want more information on evaluating the classification task checkout_ [this blogpost](https://tryolabs.com/blog/2013/03/25/why-accuracy-alone-bad-measure-classification-tasks-and-what-we-can-do-about-it/
) or [this paper](http://www.flinders.edu.au/science_engineering/fms/School-CSEM/publications/tech_reps-research_artfcts/TRRA_2007.pdf
)]



### Q9 Student Answers:
> __d)__  In the unsmoothed model performance, we notice a lot of false negatives and low acuracy. This is because there are unseen words which due to no smoothing, are making the likelihood 0 for the document to be classified, when it is actually belonging to the class.
While accuracy provides info about model's performance, it is not enough. We also need to consider precision since an email being categorized as spam when it is not is a very costly error. Recall also needs to be considered but it is not as important as precision. That is why fscore is not the appropriate metric to measure in this case because it gives equal weightage to precision and recall.



__Test/Train split__

In [49]:
# part a - test/train split (RUN THIS CELL AS IS)
!head -n 80 data/enronemail_1h.txt > data/enron_train.txt
!tail -n 20 data/enronemail_1h.txt > data/enron_test.txt
!hdfs dfs -copyFromLocal data/enron_train.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal data/enron_test.txt {HDFS_DIR}

copyFromLocal: `/user/root/HW2/enron_train.txt': File exists
copyFromLocal: `/user/root/HW2/enron_test.txt': File exists


__Training__ (Enron MNB Model _without smoothing_ )

In [50]:
# part b -  Unsmoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-model

# hadoop command
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer.py\
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/enron-model\
  -cmdenv PATH={PATH} \
  -numReduceTasks 3

# save the model locally
!mkdir NaiveBayes/Unsmoothed
!hdfs dfs -cat {HDFS_DIR}/enron-model/part-000* > NaiveBayes/Unsmoothed/NBmodel.txt

Deleted /user/root/HW2/enron-model
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob358471534606976465.jar tmpDir=null
21/06/08 01:04:34 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 01:04:34 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 01:04:34 INFO mapred.FileInputFormat: Total input paths to process : 1
21/06/08 01:04:35 INFO mapreduce.JobSubmitter: number of splits:2
21/06/08 01:04:35 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1623070339035_0033
21/06/08 01:04:35 INFO impl.YarnClientImpl: Submitted application application_1623070339035_0033
21/06/08 01:04:35 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1623070339035_0033/
21/06/08 01:04:35 INFO mapreduce.Job: Running job: job_1623070339035_0033
21/06/08 01:04:41 INFO mapreduce.Job: Job job_1623070339035_0033 running in uber mode : false
21/06/08 01:04:41 INFO mapr

In [51]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000172547666293,0.000296823983378

assistance	2.0,4.0,0.0001725476662928134,0.00029682398337785694


In [52]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,8.62738331464e-05,0.00163253190858

money	1.0,22.0,8.62738331464067e-05,0.001632531908578213


__Training__ (Enron MNB Model _with Laplace +1 smoothing_ )

In [53]:
# part b -  Smoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-model
!hdfs dfs -rm -r {HDFS_DIR}/smooth-model_part1

# hadoop command
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer_smooth.py\
  -mapper train_mapper.py \
  -reducer train_reducer_smooth.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/smooth-model_part1\
  -cmdenv PATH={PATH} \
  -numReduceTasks 3

!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -files NaiveBayes/train_second_mapper_smooth.py,NaiveBayes/train_second_reducer_smooth.py\
  -mapper train_second_mapper_smooth.py \
  -reducer train_second_reducer_smooth.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/smooth-model_part1 \
  -output {HDFS_DIR}/smooth-model\
  -cmdenv PATH={PATH} \
  -numReduceTasks 3

# save the model locally
!mkdir NaiveBayes/Smoothed
!hdfs dfs -cat {HDFS_DIR}/smooth-model/part-000* > NaiveBayes/Smoothed/NBmodel.txt

Deleted /user/root/HW2/smooth-model
Deleted /user/root/HW2/smooth-model_part1
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob953613507999419806.jar tmpDir=null
21/06/08 01:05:11 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 01:05:11 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 01:05:11 INFO mapred.FileInputFormat: Total input paths to process : 1
21/06/08 01:05:12 INFO mapreduce.JobSubmitter: number of splits:2
21/06/08 01:05:12 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1623070339035_0034
21/06/08 01:05:12 INFO impl.YarnClientImpl: Submitted application application_1623070339035_0034
21/06/08 01:05:12 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1623070339035_0034/
21/06/08 01:05:12 INFO mapreduce.Job: Running job: job_1623070339035_0034
21/06/08 01:05:18 INFO mapreduce.Job: Job job_1623070339035_0034 running in ub

In [54]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000185804533631,0.000277300205202

assistance	2.0,4.0,0.0001858045336306206,0.00027730020520215184


In [55]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,0.000123869689087,0.00127558094393

money	1.0,22.0,0.0001238696890870804,0.0012755809439298986


__Evaluation__

In [56]:
# part c - write your code in NaiveBayes/evaluation_reducer.py then RUN THIS
!chmod a+x NaiveBayes/evaluation_reducer.py

In [57]:
# part c - unit test your evaluation job on the chinese model (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py 
#!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | NaiveBayes/evaluation_reducer.py

d5	1	-8.90668134500626	-8.10769031284611	1
d6	1	-5.780743515794329	-4.179502370564408	1
d7	0	-6.591673732011658	-7.511706880737812	0
d8	0	-4.394449154674438	-5.565796731681498	0


In [58]:
# part c - Evaluate the UNSMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/unsmooth-results
!hdfs dfs -rm -r {HDFS_DIR}/unsmooth-results-part1

# hadoop job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NaiveBayes/Unsmoothed/NBmodel.txt,NaiveBayes/evaluation_reducer.py \
  -mapper classify_mapper.py \
  -reducer evaluation_reducer.py \
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/unsmooth-results \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}


# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/unsmooth-results/part-000* > NaiveBayes/Unsmoothed/results.txt

Deleted /user/root/HW2/unsmooth-results
rm: `/user/root/HW2//unsmooth-results-part1': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob934729413121623592.jar tmpDir=null
21/06/08 01:06:19 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 01:06:19 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/06/08 01:06:20 INFO mapred.FileInputFormat: Total input paths to process : 1
21/06/08 01:06:20 INFO mapreduce.JobSubmitter: number of splits:2
21/06/08 01:06:20 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1623070339035_0036
21/06/08 01:06:20 INFO impl.YarnClientImpl: Submitted application application_1623070339035_0036
21/06/08 01:06:20 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1623070339035_0036/
21/06/08 01:06:20 INFO mapreduce.Job: Running job: job_1623070339035_0036
21/06/08 01:06:27 INFO mapreduce.Job: Job jo

In [59]:
# part c - display results 
# NOTE: feel free to modify the tail commands to match the format of your results file
print('=========== UNSMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Unsmoothed/results.txt
print('=========== SMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Smoothed/results.txt

=========== UNSMOOTHED MODEL ============
# Documents:	20.0
True Positives:	1.0
True Negatives:	9.0
False Positives:	0.0
False Negatives:	10.0
Accuracy:	0.5
Precision:	1.0
Recall:	0.09090909090909091
F-Score:	0.16666666666666669
=========== SMOOTHED MODEL ============
# Documents:	20.0
True Positives:	11.0
True Negatives:	6.0
False Positives:	3.0
False Negatives:	0.0
Accuracy:	0.85
Precision:	0.7857142857142857
Recall:	1.0
F-Score:	0.88


__`EXPECTED RESULTS:`__ 
<table>
<th>Unsmoothed Model</th>
<th>Smoothed Model</th>
<tr>
<td><pre>
# Documents:	20
True Positives:	1
True Negatives:	9
False Positives:	0
False Negatives:	10
Accuracy	0.5
Precision	1.0
Recall	0.0909
F-Score	0.1666
</pre></td>
<td><pre>
# Documents:	20
True Positives:	11
True Negatives:	6
False Positives:	3
False Negatives:	0
Accuracy	0.85
Precision	0.7857
Recall	1.0
F-Score	0.88
</pre></td>
</tr>
</table>

__`NOTE:`__ _Don't be too disappointed if these seem low to you. We've trained and tested on a very very small corpus... bigger datasets coming soon!_

# Question 10: Custom Partitioning and Secondary Sort

Now that we have our model, we can analyse the results and think about future improvements.

### Q10 Tasks:

* __a) code + short response:__ Let's look at the top ten words with the highest conditional probability in `Spam` and in `Ham`. We'll do this by writing a Hadoop job that sorts the model file (`NaiveBayes/Smoothed/NBmodel.py`). Normally we'd have to run two jobs -- one that sorts on $P(word|ham)$ and another that sorts on $P(word|spam)$. However if we slighly modify the data format in the model file then we can get the top words in each class with just one job. We've written a mapper that will do just this for you. Read through __`NaiveBayes/model_sort_mapper.py`__ and then briefly explain how this mapper will allow us to partition and sort our model file. Write a Hadoop job that uses our mapper and `/bin/cat` for a reducer to partition and sort. Print out the top 10 words in each class (where 'top' == highest conditional probability).[`HINT:` _this should remind you a lot of what we did in Question 6._]


* __b) short response:__ What do you notice about the 'top words' we printed in `a`? How would increasing the smoothing parameter 'k' affect the probabilities for the top words that you identified for 'a'. How would they affect the probabilities of words that occur much more in one class than another? In summary, how does the smoothing parameter 'k' affect the bias and the variance of our model. [`NOTE:` _you do not need to code anything for this task, but if you are struggling with it you could try changing 'k' and see what happens to the test set. We don't recommend doing this exploration with the Enron data because it will be harder to see the impact with such a big vocabulary_]

### Q10 Student Answers:
> __a)__ Type your answer here!

> __b)__ Type your answer here!

In [ ]:
# part a - write your Hadoop job here (sort smoothed model on P(word|class))

# clear output directory

# hadoop job

In [ ]:
# part b - print top words in each class


### Congratulations, you have completed HW2! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/e/1FAIpQLSce9feiQeSkdP43A0ZYui1tMGIBfLfzb0rmgToQeZD9bXXX8Q/viewform